# Working with CDP Transcripts

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database and file store usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Find all transcripts

Simple query the transcript table!

In [2]:
transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
transcripts.head()

,confidence,created,event_id,file_id,transcript_id
0,0.951407,2019-07-08 01:56:18.415549,95145384-360b-46a5-9786-4a3d76ef25fb,712d2846-62d6-4ce1-8778-40300d0a7ef0,0cbb77fd-e15a-4ed9-9db9-898f42727d37
1,0.927233,2019-07-08 02:26:59.837954,d613b0d1-b5c0-4d59-9c60-0492085cbcfa,7f8a40ba-785d-4fea-af8b-4cf0c921a273,26e14fed-fdd1-4c09-aa26-a71b0692bbbf
2,0.941180,2019-07-08 01:41:49.544058,8681c031-09cf-4828-98f3-202417205a6f,2ba9df03-a005-45e9-b1ed-37ba957e6214,27d437e9-07d0-49b1-a965-4e9aa834e28c
3,0.952584,2019-07-08 01:03:21.183410,abefd4c3-759f-4d6d-8f6c-2057072d73cb,62d02b71-deeb-4d43-bd25-0e7a2d88c8c1,29df1413-6036-4f49-883d-f2804f770778
4,0.915536,2019-07-08 02:04:45.270931,15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9,900ff81f-02c2-4893-a139-5509243a6398,2eda582d-9893-4d23-aeca-b392db86b8a0


### Join file, event, and body information

While the above results are somewhat useful, we should probably merge information from the other tables too...

In [3]:
# Get the other tables
events = pd.DataFrame(db.select_rows_as_list("event"))
bodies = pd.DataFrame(db.select_rows_as_list("body"))
files = pd.DataFrame(db.select_rows_as_list("file"))

# Merge the tables
events = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
transcripts = transcripts.merge(files, left_on="file_id", right_on="file_id", suffixes=("_transcript", "_file"))
transcripts = transcripts.merge(events, left_on="event_id", right_on="event_id", suffixes=("_transcript", "_event"))
transcripts.head()

,confidence,created_transcript,event_id,file_id,transcript_id,content_type,created_file,description_transcript,filename,uri,...,created_event,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description_event,name
0,0.951407,2019-07-08 01:56:18.415549,95145384-360b-46a5-9786-4a3d76ef25fb,712d2846-62d6-4ce1-8778-40300d0a7ef0,0cbb77fd-e15a-4ed9-9db9-898f42727d37,None,2019-07-08 01:55:58.477787,None,d74142c4900f34c60be15990d0d2b315d0eaa48bbd4cf8...,gs://stg-cdp-seattle.appspot.com/d74142c4900f3...,...,2019-07-08 01:56:04.332503,2019-06-12 14:00:00,3980,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/fi...,2019-07-08 00:35:24.391795,None,Finance and Neighborhoods Committee
1,0.927233,2019-07-08 02:26:59.837954,d613b0d1-b5c0-4d59-9c60-0492085cbcfa,7f8a40ba-785d-4fea-af8b-4cf0c921a273,26e14fed-fdd1-4c09-aa26-a71b0692bbbf,None,2019-07-08 02:26:39.513874,None,ded162c1fd46fc4c70e22c2b576f4808d79acfe286fb46...,gs://stg-cdp-seattle.appspot.com/ded162c1fd46f...,...,2019-07-08 02:26:43.742074,2019-06-19 09:30:00,4003,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pl...,2019-07-08 02:26:43.482776,None,"Planning, Land Use, and Zoning Committee"
2,0.941180,2019-07-08 01:41:49.544058,8681c031-09cf-4828-98f3-202417205a6f,2ba9df03-a005-45e9-b1ed-37ba957e6214,27d437e9-07d0-49b1-a965-4e9aa834e28c,None,2019-07-08 01:41:40.307359,None,1c765c4c84649938e36fa141903992bd066dc744fb66c0...,gs://stg-cdp-seattle.appspot.com/1c765c4c84649...,...,2019-07-08 01:41:43.721244,2019-06-25 09:30:00,4009,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-07-08 01:41:43.471524,None,"Civil Rights, Utilities, Economic Development,..."
3,0.952584,2019-07-08 01:03:21.183410,abefd4c3-759f-4d6d-8f6c-2057072d73cb,62d02b71-deeb-4d43-bd25-0e7a2d88c8c1,29df1413-6036-4f49-883d-f2804f770778,None,2019-07-08 01:03:02.644955,None,76f0e3dea289f61d3c37e371d0af41cadfeef4ffa4e00c...,gs://stg-cdp-seattle.appspot.com/76f0e3dea289f...,...,2019-07-08 01:03:09.818103,2019-06-10 09:30:00,3984,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/CouncilBriefings...,http://video.seattle.gov:8080/media/council/br...,2019-07-07 22:56:33.472965,None,Council Briefing
4,0.915536,2019-07-08 02:04:45.270931,15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9,900ff81f-02c2-4893-a139-5509243a6398,2eda582d-9893-4d23-aeca-b392db86b8a0,None,2019-07-08 02:04:27.416108,None,194ba87bfce105fce73ad52e2d29b1b399d0a120790581...,gs://stg-cdp-seattle.appspot.com/194ba87bfce10...,...,2019-07-08 02:04:33.697012,2019-06-11 14:00:00,3981,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-08 02:04:33.183759,None,"Human Services, Equitable Development, and Ren..."


### Downloading and reading a transcript

A manifest is great, but you probably want to work with the transcripts locally.

In [4]:
# Download to local
save_path = fs.download_file(transcripts.loc[3]["filename"])
save_path

PosixPath('/home/maxfield/active/cdp/cdptools/examples/76f0e3dea289f61d3c37e371d0af41cadfeef4ffa4e00c6f53c2426c12ab70db_ts_sentences_transcript_0.txt')

In [5]:
# Read the transcript
import json
with open(save_path, "r") as read_in:
    sentences = json.load(read_in)
    for s in sentences[:3]:
        print(s)

{'sentence': 'Good morning.', 'start_time': 18.4, 'end_time': 20.0}
{'sentence': 'Thank you for being here for a regularly scheduled meeting.', 'start_time': 20.0, 'end_time': 22.0}
{'sentence': "I'm just passing on copies of our minutes.", 'start_time': 22.0, 'end_time': 24.6}


### Handling multiple transcripts for a single event

A single event may have multiple versions of a transcripts. In this case you can decide how you want to filter them down, but generally, a decent solution is to group by `id_event`  then choose the most recent. You can do this with `pandas.Series.idxmax()`.

[stackoverflow](https://stackoverflow.com/questions/10202570/pandas-dataframe-find-row-where-values-for-column-is-maximal)

In [6]:
# Group rows
most_recent_transcripts = []
grouped = transcripts.groupby("event_id")
for name, group in grouped:
    most_recent = group.loc[group["created_transcript"].idxmax()]
    most_recent_transcripts.append(most_recent)
    
most_recent = pd.DataFrame(most_recent_transcripts)
most_recent.head()

,confidence,created_transcript,event_id,file_id,transcript_id,content_type,created_file,description_transcript,filename,uri,...,created_event,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri,created_body,description_event,name
4,0.915536,2019-07-08 02:04:45.270931,15ce0a20-3688-4ebd-bf3f-24f6e8d12ad9,900ff81f-02c2-4893-a139-5509243a6398,2eda582d-9893-4d23-aeca-b392db86b8a0,None,2019-07-08 02:04:27.416108,None,194ba87bfce105fce73ad52e2d29b1b399d0a120790581...,gs://stg-cdp-seattle.appspot.com/194ba87bfce10...,...,2019-07-08 02:04:33.697012,2019-06-11 14:00:00,3981,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-08 02:04:33.183759,None,"Human Services, Equitable Development, and Ren..."
22,0.942335,2019-07-08 01:50:12.463640,3571c871-6f7d-41b5-85d1-ced0589f9220,03e9b46f-d7d4-4bd0-888e-4d2d0cbc2ce3,ce3f4436-9d1c-40bc-a013-0b6fc05e16e0,None,2019-07-08 01:49:53.623882,None,cee27aca34b4e27f66dde5e7e0c901b600f35a3e209b61...,gs://stg-cdp-seattle.appspot.com/cee27aca34b4e...,...,2019-07-08 01:50:00.679099,2019-06-11 09:30:00,3982,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/ci...,2019-07-08 01:41:43.471524,None,"Civil Rights, Utilities, Economic Development,..."
16,0.937972,2019-07-08 01:23:16.383044,44af0ef0-cdfc-4cf1-924f-913065a0a21e,b683c0dc-e38a-4e8d-ac5a-28f9736035f8,9092f736-9256-47dd-bcba-92722679fc17,None,2019-07-08 01:23:04.125599,None,ed9e1b594c1da4b89d55d7ee6446d82f1042b70ac0d4ed...,gs://stg-cdp-seattle.appspot.com/ed9e1b594c1da...,...,2019-07-08 01:23:09.749805,2019-06-24 09:30:00,3987,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/pa...,2019-07-07 22:56:33.472965,None,Council Briefing
8,0.942447,2019-07-08 02:24:00.297071,4661ee79-b85f-44b2-b784-9aa97372b35c,bae8febc-462c-48a5-a7cc-d42cf2439bfe,34385d05-e10d-4d9e-87c8-30604d1fddf6,None,2019-07-08 02:23:48.472120,None,011c9aaaf743549497c685cbcd52afc9d3a55226eff359...,gs://stg-cdp-seattle.appspot.com/011c9aaaf7435...,...,2019-07-08 02:23:53.312312,2019-06-25 14:00:00,4012,https://seattle.legistar.com/MeetingDetail.asp...,None,http://www.seattlechannel.org/mayor-and-counci...,http://video.seattle.gov:8080/media/council/hu...,2019-07-08 02:04:33.183759,None,"Human Services, Equitable Development, and Ren..."
24,0.917240,2019-07-08 00:53:31.586694,56af907c-51cb-4364-bbbf-e686e1244f27,754a5973-d0fb-48f2-a6f1-c320edef611a,dab2777a-f102-4bef-a9af-dd248ff28298,None,2019-07-08 00:52:02.385447,None,99d280be442e2a46ddf643eab10a347c87bcfabc0214ee...,gs://stg-cdp-seattle.appspot.com/99d280be442e2...,...,2019-07-08 00:52:09.898324,2019-06-17 14:00:00,4002,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...,2019-07-07 23:46:03.311538,None,City Council


### A function to download the most recent transcripts

To make gathering the most recent transcripts easier, let's wrap this example up with a function to do all this for us.

**Note:** The function defined below is available in the `cdptools.utils` module.

`from cdptools.utils import download_most_recent_transcripts`

In [7]:
from pathlib import Path
from typing import Optional, Union

from cdptools.databases import Database
from cdptools.file_stores import FileStore


def download_most_recent_transcripts(db: Database, fs: FileStore, save_dir: Optional[Union[str, Path]] = None) -> Path:
    """
    Download the most recent versions of event transcripts.

    :param db: An already initialized `Database` object to query against.
    :param fs: An alreay initialized `FileStore` object to query against.
    :param save_dir: Path to a directory to save the transcripts and the dataset manifest.
    :return: Fully resolved path where transcripts and dataset manifest were stored.
    """
    # Use current directory is None provided
    if save_dir is None:
        save_dir = "."
    
    # Resolve save directory
    save_dir = Path(save_dir).resolve()
    
    # Make the save directory if not already exists
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Get transcript dataset
    transcripts = pd.DataFrame(db.select_rows_as_list("transcript"))
    events = pd.DataFrame(db.select_rows_as_list("event"))
    bodies = pd.DataFrame(db.select_rows_as_list("body"))
    files = pd.DataFrame(db.select_rows_as_list("file"))
    events = events.merge(bodies, left_on="body_id", right_on="body_id", suffixes=("_event", "_body"))
    transcripts = transcripts.merge(files, left_on="file_id", right_on="file_id", suffixes=("_transcript", "_file"))
    transcripts = transcripts.merge(events, left_on="event_id", right_on="event_id", suffixes=("_transcript", "_event"))
    
    # Group
    most_recent_transcripts = []
    grouped = transcripts.groupby("event_id")
    for name, group in grouped:
        most_recent = group.loc[group["created_transcript"].idxmax()]
        most_recent_transcripts.append(most_recent)
    
    most_recent = pd.DataFrame(most_recent_transcripts)
    
    # Begin storage
    most_recent.apply(
        lambda r: fs.download_file(r["filename"], save_dir / r["filename"], overwrite=True), 
        axis=1
    )
    
    # Write manifest
    most_recent.to_csv(save_dir / "transcript_manifest.csv", index=False)
    
    return save_dir

In [8]:
save_dir = download_most_recent_transcripts(db, fs, "most_recent_seattle_transcripts")
save_dir

PosixPath('/home/maxfield/active/cdp/cdptools/examples/most_recent_seattle_transcripts')